# This notebook will populate the .ini files with the injection parameters

In [1]:
import bilby as bb
import numpy as np
import gwcosmo as gwc
import pandas as pd
import GCR
import GCRCatalogs as GCRCat
import matplotlib.pyplot as plt
import os
from astropy.time import Time

In [6]:
nsbhIniTemplatePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/slurmJobs/prod/template_acceptance-walk_prod_NSBH_beforeModifications.ini"
bbhIniTemplatePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/slurmJobs/prod/template_acceptance-walk_prod_BBH_beforeModifications.ini"

## The following should be appended to the existing template .ini files:
- `label`: best to use the galaxyID_iterator based on some galaxies having multiple CBC's
- `outdir`: `os.path.join(basePath,label)`
- `injection_dict`: pulled directly from the entry in the CBC catalog. Note that the keys needed are 'mass_ratio', 'chirp_mass', 'luminosity_distance', 'dec','ra', 'theta_jn', 'psi', 'phase', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', and 'geocent_time'. Note that geocent time should be set to zero

In [51]:
basePathBBH = "/pscratch/sd/s/seanmacb/BBH_GalaxyIDs"
basePathNSBH = "/pscratch/sd/s/seanmacb/NSBH_GalaxyIDs"

- Set `outdir` to basePath[NSBH.BBH]/batch_[0..11]/{galaxyID_iterator}
- Set `label` to `galaxyID_iterator`
- Another thing to look at - automatically determining the duration 

In [110]:
def append_to_template(template_path, new_lines, output_path):
    """
    Read a template file, append lines, and write to a new file.

    Parameters:
    - template_path (str): Path to the original template file.
    - new_lines (list of str): Lines to append to the file.
    - output_path (str): Path to save the new file.
    """
    # Read the original template
    with open(template_path, "r") as file:
        content = file.read()

    # Add the new lines
    appended_content = content + "\n" + "\n".join(new_lines) + "\n"

    # Write the new content to the output file
    with open(output_path, "w") as file:
        file.write(appended_content)
    return output_path


def formatLines(galaxyRow, basePath, df):
    """
    Read the galaxyRow, basePath, the input dataframe, and returns a list of formatted lines that match the .ini syntax

    Parameters:
    - galaxyRow (Pandas Series): The row entry of the CBC in question, taken directly from the mock CBC catalog
    - basePath (str): The base path to the directory where the ini file will be saved
    - df (Pandas dataframe): The dataframe of the mock CBC catalog
    """
    keys = [
        "mass_ratio",
        "chirp_mass",
        "luminosity_distance",
        "dec",
        "ra",
        "theta_jn",
        "psi",
        "phase",
        "a_1",
        "a_2",
        "tilt_1",
        "tilt_2",
        "phi_12",
        "phi_jl",
        "geocent_time",
    ]  # Injection keys
    dfKeys = [
        "mass_ratio",
        "chirp_mass",
        "luminosity_distance",
        "mdec",
        "mra",
        "theta_jn",
        "psi",
        "phase",
        "a_1",
        "a_2",
        "tilt_1",
        "tilt_2",
        "phi_12",
        "phi_jl",
        "geocent_time",
    ]  # Dataframe keys

    # Format the first line, related to label
    galaxyID = int(galaxyRow["galaxyID"])

    a = np.sort(df[df["galaxyID"] == galaxyRow["galaxyID"]].index)
    iterator = np.argwhere(a == galaxyRow.name)[0][0]

    line1 = "label={}_{}".format(galaxyID, iterator)

    # Format the second line, related to the outdir
    line2 = "outdir={}/{}_{}".format(basePath, galaxyID, iterator)

    # Format the third line, related to the injection dict
    # indexedRow = galaxyRow[dfKeys]

    injection_dict = {}

    for k1, k2 in zip(keys, dfKeys):
        try:
            injection_dict[k1] = testRow[k2]
        except:
            injection_dict[k1] = 0  # This is the geocent_time case=

    line3 = "injection_dict={}".format(injection_dict)

    return line1, line2, line3


"""
Utilities to compute component masses from ratio and chirp mass
While chirp!=m_1+m2, it's an approximation for component mass, using componentM ~ 2.5*chirp as the upper bound on total component mass
"""


def getMasses(ratio, chirp):
    return getMassA(ratio, chirp), getMassB(ratio, chirp)


def getMassA(ratio, chirp):
    return chirp * ratio


def getMassB(ratio, chirp):
    return chirp - chirp * ratio

In [116]:
outPath = "/pscratch/sd/s/seanmacb/iniTest"
lines = formatLines(bbhDF.loc[0], outPath, bbhDF)
outFile = lines[0].split("=")[1] + ".ini"
append_to_template(bbhIniTemplatePath, lines, os.path.join(outPath, outFile))

'/pscratch/sd/s/seanmacb/iniTest/10070200392465_0.ini'

## It works! Now it's just time to select the subsets of BBH and NSBH events that meet the following criteria

- Uniform weighting, fiducial cosmology
- Uniform weighting, skewed cosmology
- Luminosity weighting, fiducial cosmology
- SFR/Stellar mass weighting
- GW host outside of galaxy catalog, GW localization volume inside of catalog footprint
- GW host inside of galaxy catalog, GW localization volume inside of catalog footprint
- GW host inside of galaxy catalog, GW localization volume partially outside of catalog footprint
- Photo-z precision effect on cosmological results (2 scenarios)
- Photometric completeness effects (2 scenarios)

# Galaxy Catalog conditions

### Fiducial cosmology, uniform, luminosity, or SFR/stellar mass weighting

These three scenarios only require a difference in the gwCosmo analysis configuration, but can rely on the same CBC samples

The galaxy catalog will stay the same

The CBC dataset will stay the same

This will be the reference dataset of CBC and galaxies

The outcome of these three studies will be convergence towards the optimal analysis configuration

### Photo-z precision effect on cosmological results (2 scenarios)

More precise, less precise

This is on the **galaxy catalog** end, so there is nothing that needs to be changed for the GW injections

The CBC dataset will be the reference

The analysis configuration will be optimal for this scenario

### Photometric completeness effects (2 scenarios)

Less complete, more complete (or even lesser complete?)

This is on the **galaxy catalog** end, so there is nothing that needs to be changed for the GW injections

The CBC dataset will be the reference

The analysis configuration will be optimal for this scenario

# Skewed Cosmology

### Uniform weighting, skewed cosmology

This scenario involves using a skewed value of $H_0$ for CBC injections. We will choose a value of 65, compared to the 71 used in skySim5000

The CBC dataset will be skewed

The Galaxy catalog will be the reference

The analysis configuration will be optimal for this scenario

## GW selection effects

### GW host inside galaxy catalog, GW localization volume inside of catalog footprint

This is will be the optimal analysis configurations, reference datasets

### GW host inside of galaxy catalog, GW localization volume outside of catalog footprint

Have the host on the edge of the region cut of the effective galaxy catalog

The analysis configuration will be optimal for this scenario

The CBC dataset will be modified to support the region cut **OR** The galaxy catalog will be modified for the region cut. Whichever dataset is not modified, will be the reference.

### GW host outside of galaxy catalog, GW localization volume inside of catalog footprint

For this, it is best to use the full skySim5000 catalog, with an arbitrary region cut to reject the CBC host while still maintaining significant area of the injection in the galaxy footprint, or the host manually removed

The CBC dataset will be modified to support the region cut **OR** The galaxy catalog will be modified for the region cut. Whichever dataset is not modified, will be the reference.

The analysis configuration will be optimal for this scenario